In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import ResNet50
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Input
from keras.models import Sequential

Using TensorFlow backend.


## Reading and formatting files for kaggle

In [2]:
def forKaggle1(x):
    y = "../input/train"+x[9:]
    return y

def forKaggle2(x):
    y = "../input/valid"+x[9:]
    return y

def formatting1(string):
    y = pd.read_csv(string, names=['filename', 'abnormal'])
    y.filename = y.filename.apply(lambda x: forKaggle1(x))
    y.abnormal = y.abnormal.astype('int').astype('str').astype('category')
    return y

def formatting2(string):
    y = pd.read_csv(string, names=['filename', 'abnormal'])
    y.filename = y.filename.apply(lambda x: forKaggle2(x))
    y.abnormal = y.abnormal.astype('int').astype('str').astype('category')
    return y

In [3]:
humerus_train = formatting1("../input/trainset_humerus.csv")
humerus_val = formatting2("../input/testset_humerus.csv")

elbow_train = formatting1("../input/trainset_elbow.csv")
elbow_val = formatting2("../input/testset_elbow.csv")

finger_train = formatting1("../input/trainset_finger.csv")
finger_val = formatting2("../input/testset_finger.csv")

forearm_train = formatting1("../input/trainset_forearm.csv")
forearm_val = formatting2("../input/testset_forearm.csv")

hand_train = formatting1("../input/trainset_hand.csv")
hand_val = formatting2("../input/testset_hand.csv")

shoulder_train = formatting1("../input/trainset_shoulder.csv")
shoulder_val = formatting2("../input/testset_shoulder.csv")

wrist_train = formatting1("../input/trainset_wrist.csv")
wrist_val = formatting2("../input/testset_wrist.csv")

In [4]:
img_size = 224
b_size = 64

def train_gen(DATAFRAME):
    train_data_gen = ImageDataGenerator(rescale=1./255, rotation_range = 45, horizontal_flip = True)
    train_generator = train_data_gen.flow_from_dataframe(dataframe = DATAFRAME, directory = None,
                                        x_col = 'filename', y_col = 'abnormal', target_size=(img_size, img_size),
                                        batch_size=b_size, shuffle = False, color_mode = 'rgb',
                                        class_mode = 'binary', drop_duplicates=False)
    return train_generator

def val_gen(DATAFRAME):
    val_datagen = ImageDataGenerator(rescale=1. / 255)
    val_generator = val_datagen.flow_from_dataframe(dataframe = DATAFRAME, directory = None,
                                        x_col = 'filename', y_col = 'abnormal', target_size=(img_size, img_size),
                                        batch_size=b_size, shuffle = False, color_mode = 'rgb',
                                        class_mode = 'binary', drop_duplicates=False)
    return val_generator

def init_model(model_name):
    #from keras import losses
    #from keras.callbacks import EarlyStopping, ModelCheckpoint
    stop = EarlyStopping(patience=30)
    checkpoint = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
    return stop, checkpoint

resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(img_size,img_size,3))
print('Done')

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Done


## Training

In [5]:
ep = 40

train_generator = train_gen(humerus_train)
val_generator = val_gen(humerus_val)

humerus_model = Sequential()
humerus_model.add(resnet)
humerus_model.add(Flatten())
humerus_model.add(Dense(1, activation='softmax'))
stop, checkpoint = init_model('humerus_model.h5')
humerus_model.compile(optimizer=Adam(lr=1e-3, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

humerus_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

humerus_model.save('humerus_model.h5')

Found 1272 images belonging to 2 classes.
Found 288 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
20/20 [==============================] - 35s 2s/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 2/40
20/20 [==============================] - 19s 936ms/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 3/40
20/20 [==============================] - 20s 991ms/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 4/40
20/20 [==============================] - 20s 996ms/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 5/40
20/20 [==============================] - 20s 993ms/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 6/40
20/20 [==============================] - 20s 998ms/step - loss: 8.4173 - acc: 0.4720 - val_loss: 8.1926 - val_acc: 0.4861
Epoch 7/40
20/20 [==============================] - 20s 1s/step - loss: 8.

In [6]:
train_generator = train_gen(elbow_train)
val_generator = val_gen(elbow_val)

elbow_model = Sequential()
elbow_model.add(resnet)
elbow_model.add(Flatten())
elbow_model.add(Dense(1, activation='softmax'))
stop, checkpoint = init_model('elbow_model.h5')
elbow_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

elbow_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

elbow_model.save('elbow_model.h5')

Found 4931 images belonging to 2 classes.
Found 465 images belonging to 2 classes.
Epoch 1/40
78/78 [==============================] - 96s 1s/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 2/40
78/78 [==============================] - 74s 954ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 3/40
78/78 [==============================] - 76s 971ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 4/40
78/78 [==============================] - 76s 976ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 5/40
78/78 [==============================] - 76s 970ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 6/40
78/78 [==============================] - 76s 979ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_acc: 0.5075
Epoch 7/40
78/78 [==============================] - 76s 971ms/step - loss: 9.4041 - acc: 0.4101 - val_loss: 7.8512 - val_a

In [7]:
train_generator = train_gen(finger_train)
val_generator = val_gen(finger_val)

finger_model = Sequential()
finger_model.add(resnet)
finger_model.add(Flatten())
finger_model.add(Dense(1, activation='softmax'))
stop, checkpoint = init_model('finger_model.h5')
finger_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

finger_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

finger_model.save('finger_model.h5')

Found 5106 images belonging to 2 classes.
Found 461 images belonging to 2 classes.
Epoch 1/40
80/80 [==============================] - 100s 1s/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 2/40
80/80 [==============================] - 75s 936ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 3/40
80/80 [==============================] - 77s 958ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 4/40
80/80 [==============================] - 77s 962ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 5/40
80/80 [==============================] - 77s 961ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 6/40
80/80 [==============================] - 77s 957ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_acc: 0.5640
Epoch 7/40
80/80 [==============================] - 76s 954ms/step - loss: 9.5901 - acc: 0.3985 - val_loss: 6.9510 - val_

In [8]:
train_generator = train_gen(forearm_train)
val_generator = val_gen(forearm_val)

forearm_model = Sequential()
forearm_model.add(resnet)
forearm_model.add(Flatten())
forearm_model.add(Dense(1, activation='softmax'))
stop, checkpoint = init_model('forearm_model.h5')
forearm_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

forearm_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

forearm_model.save('forearm_model.h5')

Found 1825 images belonging to 2 classes.
Found 301 images belonging to 2 classes.
Epoch 1/40
29/29 [==============================] - 49s 2s/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 2/40
29/29 [==============================] - 27s 927ms/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 3/40
29/29 [==============================] - 29s 1s/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 4/40
29/29 [==============================] - 29s 1000ms/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 5/40
29/29 [==============================] - 29s 994ms/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 6/40
29/29 [==============================] - 29s 1000ms/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 - val_acc: 0.5150
Epoch 7/40
29/29 [==============================] - 29s 993ms/step - loss: 10.1755 - acc: 0.3617 - val_loss: 7.7329 -

In [9]:
train_generator = train_gen(hand_train)
val_generator = val_gen(hand_val)

hand_model = Sequential()
hand_model.add(resnet)
hand_model.add(Flatten())
hand_model.add(Dense(1, activation='softmax'))
hand_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
stop, checkpoint = init_model('hand_model.h5')

hand_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

hand_model.save('hand_model.h5')

Found 5543 images belonging to 2 classes.
Found 460 images belonging to 2 classes.
Epoch 1/40
87/87 [==============================] - 117s 1s/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 2/40
87/87 [==============================] - 86s 984ms/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 3/40
87/87 [==============================] - 88s 1s/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 4/40
87/87 [==============================] - 87s 999ms/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 5/40
87/87 [==============================] - 86s 988ms/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 6/40
87/87 [==============================] - 86s 994ms/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - val_acc: 0.4304
Epoch 7/40
87/87 [==============================] - 86s 986ms/step - loss: 11.4755 - acc: 0.2802 - val_loss: 9.0802 - 

In [10]:
train_generator = train_gen(shoulder_train)
val_generator = val_gen(shoulder_val)

shoulder_model = Sequential()
shoulder_model.add(resnet)
shoulder_model.add(Flatten())
shoulder_model.add(Dense(1, activation='softmax'))
shoulder_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
stop, checkpoint = init_model('shoulder_model.h5')

shoulder_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

shoulder_model.save('shoulder_model.h5')

Found 8379 images belonging to 2 classes.
Found 563 images belonging to 2 classes.
Epoch 1/40
131/131 [==============================] - 167s 1s/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 2/40
131/131 [==============================] - 125s 956ms/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 3/40
131/131 [==============================] - 128s 979ms/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 4/40
131/131 [==============================] - 128s 976ms/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 5/40
131/131 [==============================] - 127s 966ms/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 6/40
131/131 [==============================] - 127s 970ms/step - loss: 7.6612 - acc: 0.5194 - val_loss: 6.9093 - val_acc: 0.5666
Epoch 7/40
131/131 [==============================] - 127s 969ms/step - loss: 7.6612 - acc: 0.5194 - val

In [11]:
train_generator = train_gen(wrist_train)
val_generator = val_gen(wrist_val)

wrist_model = Sequential()
wrist_model.add(resnet)
wrist_model.add(Flatten())
wrist_model.add(Dense(1, activation='softmax'))
wrist_model.compile(optimizer=Adam(lr=0.01, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
stop, checkpoint = init_model('wrist_model.h5')

wrist_model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=ep, 
                            validation_data=val_generator, validation_steps=len(val_generator),
                            workers=1, use_multiprocessing=False)#, callbacks = [stop, checkpoint])

wrist_model.save('wrist_model.h5')

Found 9752 images belonging to 2 classes.
Found 659 images belonging to 2 classes.
Epoch 1/40
153/153 [==============================] - 202s 1s/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 2/40
153/153 [==============================] - 150s 978ms/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 3/40
153/153 [==============================] - 150s 984ms/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 4/40
153/153 [==============================] - 153s 998ms/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 5/40
153/153 [==============================] - 153s 998ms/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 6/40
153/153 [==============================] - 154s 1s/step - loss: 8.9853 - acc: 0.4364 - val_loss: 7.7172 - val_acc: 0.5159
Epoch 7/40
153/153 [==============================] - 152s 996ms/step - loss: 8.9853 - acc: 0.4364 - val_lo